In [6]:
# Instalación de Beautiful Soup
!pip install beautifulsoup4
from bs4 import BeautifulSoup as bs

In [7]:
# Instalacion de Requests
!pip install requests
import requests

# Elegir un sitio web del que queremos extraer datos, 
main_url = "http://books.toscrape.com/index.html"

# Solicitud GET utilizando "requests"
res = requests.get(main_url)

In [8]:
# Obtenemos el url para la paginacion(50)
def getAndParseURL(url):
    res = requests.get(url)
    soup = bs(res.text, 'html.parser')
    return(soup)

In [9]:
# Conseguimos url de los libros(1000)
def getBooksURLs(url):
    soup = getAndParseURL(url)
    # Eliminamos la parte index.html
    return(
        ["/".join(url.split("/")[:-1]) + "/" + x.div.a.get('href') 
         for x in soup.findAll(
             "article", class_ = "product_pod"
          )
        ]
    )

In [10]:
# Almacenar todos los resultados en una lista
import re 

pages_urls = [main_url]
soup = getAndParseURL(pages_urls[0])

# Aqui obtendremos los links de la paginacion de la app
# despues obtendremos los datos de cada uno de los links
while len(soup.findAll("a", href=re.compile("page"))) == 2 or len(pages_urls) == 1:
    new_url = "/".join(pages_urls[-1].split("/")[:-1]) + "/" + 
    soup.findAll("a", href=re.compile("page"))[-1].get("href")
    
    # agregamos url a la lista
    pages_urls.append(new_url)
    
    # analizar la página siguiente
    soup = getAndParseURL(new_url)
    
# resultados
print(str(len(pages_urls)) + " URL obtenidas")
pages_urls[:5]

50 URL obtenidas


['http://books.toscrape.com/index.html',
 'http://books.toscrape.com/catalogue/page-2.html',
 'http://books.toscrape.com/catalogue/page-3.html',
 'http://books.toscrape.com/catalogue/page-4.html',
 'http://books.toscrape.com/catalogue/page-5.html']

In [11]:
#Aqui guardaremos los datos de las 50 paginas de la paginacion
# Obj: debe obtener los 1000 resultados
booksURLs = []

for page in pages_urls:  #las 50 paginas

    #obtenemos los links
    booksURLs.extend(getBooksURLs(page))
    
print(str(len(booksURLs)) + " URL obtenidas")
booksURLs[:5]

1000 URL obtenidas


['http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html',
 'http://books.toscrape.com/catalogue/tipping-the-velvet_999/index.html',
 'http://books.toscrape.com/catalogue/soumission_998/index.html',
 'http://books.toscrape.com/catalogue/sharp-objects_997/index.html',
 'http://books.toscrape.com/catalogue/sapiens-a-brief-history-of-humankind_996/index.html']

In [12]:
#Ya hemos obtenido los links de los 1000 paginas
# ahora corresponde organizarla segun los datos solicitados

# LA EJECUCION DE ESTE CODIGO PUEDE DEMORAR,
# RECOMIENTO MIRAR POR LA VENTANA MIENTRAS ESPERA :P

import pandas as pd

# -------ENTRADA-------
title = []
prices = []
stock = []
categories = []
cover = []

#Product Description
upc = []
product_type = []
price_excl= []
price_incl= []
tax= []
availability= []
reviews= []

# -------PROCESO-------
# recopilar datos para la URL de cada libro
for url in booksURLs:
    soup = getAndParseURL(url)

    # Titulo del libro -> div con clase "product_main" y al h1
    title.append(
        soup.find("div", class_ = re.compile("product_main")
    ).h1.text)

    # Precio -> p con clase "price_color", [2:] incluye $
    prices.append(soup.find(
        "p", class_ = "price_color").text[2:]
    )


    # Categoria -> links de la categoria
    categories.append(soup.find(
        "a", href = re.compile("../category/books/")
    ).get("href").split("/")[3])


    # stock -> p con clase y transformamos a texto
    stock.append(re.sub("[^0-9]", "", soup.find(
        "p", class_ = "instock availability").text)
    )

    #cover: url de caratula -> vamos aindex -> img/src
    cover.append(url.replace("index.html", "") + soup.find("img").get("src"))

    # UPC --> buscamos en la table
    upc.append(soup.find(string="UPC").find_next('td').contents[0])

    # Tipo de producto
    product_type.append(soup.find(string="Product Type").find_next('td').contents[0])

    #price_excl
    price_excl.append(soup.find(string="Price (excl. tax)").find_next('td').text[2:])

    #Price (incl. tax)
    price_incl.append(soup.find(string="Price (incl. tax)").find_next('td').text[2:])

    # Impuesto
    tax.append(soup.find(string="Tax").find_next('td').text[2:])

    # Disponibilidad
    availability.append(soup.find(string="Availability").find_next('td').contents[0])

    # Reseñas
    reviews.append(soup.find(string="Number of reviews").find_next('td').contents[0])

    
    
# Agregamos a pandas para ver mejor
scraped_data = pd.DataFrame({
    'Title': title, 
    'Price': prices, 
    'Stock': stock,
    'Category': categories,
    'Cover': cover,
    'UPC': upc,
    'Product Type': product_type,
    'Price (excl. tax)': price_excl,
    'Price (incl. tax)': price_incl,
    'Tax': tax,
    'Availability': availability,
    'Number of reviews': reviews
    
})
scraped_data.head()

,Title,Price,Stock,Category,Cover,UPC,Product Type,Price (excl. tax),Price (incl. tax),Tax,Availability,Number of reviews
0,A Light in the Attic,51.77,22,poetry_23,http://books.toscrape.com/catalogue/a-light-in...,a897fe39b1053632,Books,51.77,51.77,0.00,In stock (22 available),0
1,Tipping the Velvet,53.74,20,historical-fiction_4,http://books.toscrape.com/catalogue/tipping-th...,90fa61229261140a,Books,53.74,53.74,0.00,In stock (20 available),0
2,Soumission,50.10,20,fiction_10,http://books.toscrape.com/catalogue/soumission...,6957f44c3847a760,Books,50.10,50.10,0.00,In stock (20 available),0
3,Sharp Objects,47.82,20,mystery_3,http://books.toscrape.com/catalogue/sharp-obje...,e00eb4fd7b871a48,Books,47.82,47.82,0.00,In stock (20 available),0
4,Sapiens: A Brief History of Humankind,54.23,20,history_32,http://books.toscrape.com/catalogue/sapiens-a-...,4165285e1663650f,Books,54.23,54.23,0.00,In stock (20 available),0


In [13]:
# verfificamos los links de cover -> debe ser la caratula del libro
cover[:5]

['http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/../../media/cache/fe/72/fe72f0532301ec28892ae79a629a293c.jpg',
 'http://books.toscrape.com/catalogue/tipping-the-velvet_999/../../media/cache/08/e9/08e94f3731d7d6b760dfbfbc02ca5c62.jpg',
 'http://books.toscrape.com/catalogue/soumission_998/../../media/cache/ee/cf/eecfe998905e455df12064dba399c075.jpg',
 'http://books.toscrape.com/catalogue/sharp-objects_997/../../media/cache/c0/59/c05972805aa7201171b8fc71a5b00292.jpg',
 'http://books.toscrape.com/catalogue/sapiens-a-brief-history-of-humankind_996/../../media/cache/ce/5f/ce5f052c65cc963cf4422be096e915c9.jpg']

In [15]:
# Exportar df a csv
scraped_data.to_csv(
    'data_books.csv', 
    index=False, sep=';',
    encoding='utf-8'
)

**Linkografia**

* https://towardsdatascience.com/an-introduction-to-web-scraping-with-python-a2601e8619e5